In [33]:
import pandas as pd
import re

# Funzione per estrarre i temi da V2Themes
def extract_themes(v2themes):
    if pd.isna(v2themes):  # Gestisce valori mancanti
        return []
    themes = v2themes.split(';')  # Divide i temi separati da ';'
    themes = [re.sub(r',.*', '', theme) for theme in themes]  # Rimuove il peso (tutto dopo la virgola)
    return themes

df_themes = pd.read_csv('../Data/gkg_myanmar_filtered.csv')

df_themes['theme'] = df_themes['V2Themes'].apply(extract_themes)
df_themes = df_themes['theme'].explode('theme')  # Crea una riga per ogni tema
# Conta i temi unici
theme_counts = df_themes.value_counts().reset_index()
theme_counts.columns = ['theme', 'count']

theme_counts.to_csv('../Data/Themes.csv', index=False)